In [25]:
import pandas as pd
import hopsworks
from datetime import datetime, timedelta, timezone
import util
import json
import warnings
warnings.filterwarnings("ignore")

Will download yesterdays air quality and weather and will then forecast the weather for the next 10 days

In [26]:
import dotenv

dotenv.load_dotenv()

True

In [27]:
project = hopsworks.login(project="dbillLab1")
fs = project.get_feature_store() 
secrets = util.secrets_api(project.name)

# This line will fail if you have not registered the AQI_API_KEY as a secret in Hopsworks
AQI_API_KEY = secrets.get_secret("AQI_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.today()
location_str

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169559
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


'{"country": "sweden", "city": "stockholm", "street": "stockholm-st-eriksgatan-83", "aqicn_url": "https://api.waqi.info/feed/@10523", "latitude": 59.33, "longitude": 18.07}'

In [28]:
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

In [29]:
data_frame=air_quality_fg.read().sort_values(by=['date'])
data_frame

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.48s) 


,date,pm25,country,city,street,url,pm25_1_day_before,pm25_2_days_before,pm25_3_days_before,pm25_average_before
363,2018-04-12 00:00:00+00:00,26.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523,23.0,14.0,55.0,30.666666
1554,2018-04-13 00:00:00+00:00,33.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523,26.0,23.0,14.0,21.000000
1211,2018-04-14 00:00:00+00:00,37.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523,33.0,26.0,23.0,27.333334
463,2018-04-15 00:00:00+00:00,64.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523,37.0,33.0,26.0,32.000000
111,2018-04-16 00:00:00+00:00,70.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523,64.0,37.0,33.0,44.666668
...,...,...,...,...,...,...,...,...,...,...
487,2024-11-13 00:00:00+00:00,21.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523,21.0,20.0,10.0,17.000000
379,2024-11-14 00:00:00+00:00,25.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523,21.0,21.0,20.0,20.666666
1626,2024-11-15 00:00:00+00:00,22.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523,25.0,21.0,21.0,22.333334
2138,2024-11-16 00:00:00+00:00,13.0,sweden,stockholm,stockholm-st-eriksgatan-83,https://api.waqi.info/feed/@10523,22.0,25.0,21.0,22.666666


Get the lagging air quality

In [30]:
today=datetime.today() - timedelta(0)
today=today.replace(hour=0, minute=0, second=0, microsecond=0, tzinfo=timezone.utc)
one_day_before=today-timedelta(days=1)
two_days_before= today-timedelta(days=2)
three_days_before= today-timedelta(days=3)

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQI_API_KEY)

aq_today_df["pm25_1_day_before"]=data_frame.loc[data_frame["date"]==one_day_before,"pm25"].values
aq_today_df["pm25_2_days_before"]=data_frame.loc[data_frame["date"]==two_days_before,"pm25"].values
aq_today_df["pm25_3_days_before"]=data_frame.loc[data_frame["date"]==three_days_before,"pm25"].values

aq_today_df["pm25_average_before"]=aq_today_df[["pm25_1_day_before", "pm25_2_days_before","pm25_3_days_before"]].mean(axis=1)
aq_today_df


,pm25,country,city,street,date,url,pm25_1_day_before,pm25_2_days_before,pm25_3_days_before,pm25_average_before
0,15.0,sweden,stockholm,stockholm-st-eriksgatan-83,2024-11-17 00:00:00+00:00,https://api.waqi.info/feed/@10523,13.0,22.0,25.0,20.0


In [31]:
aq_today_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   pm25                 1 non-null      float32            
 1   country              1 non-null      object             
 2   city                 1 non-null      object             
 3   street               1 non-null      object             
 4   date                 1 non-null      datetime64[us, UTC]
 5   url                  1 non-null      object             
 6   pm25_1_day_before    1 non-null      float32            
 7   pm25_2_days_before   1 non-null      float32            
 8   pm25_3_days_before   1 non-null      float32            
 9   pm25_average_before  1 non-null      float32            
dtypes: datetime64[us, UTC](1), float32(5), object(4)
memory usage: 188.0+ bytes


In [32]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# Gets forecast for next 10 days
# Replace  hourly forecasts with a single daily forecast
# Get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])     #To date format

daily_df['city'] = city
daily_df

Coordinates 59.25°N 18.0°E
Elevation 18.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2024-11-17,6.45,0.0,18.388212,246.948669,stockholm
1,2024-11-18,2.50,0.0,10.895576,277.594543,stockholm
2,2024-11-19,0.80,0.0,2.620839,105.945465,stockholm
3,2024-11-20,1.25,0.4,25.932495,13.650437,stockholm
4,2024-11-21,0.75,0.2,17.102840,310.732025,stockholm
5,2024-11-22,-3.90,0.1,14.113653,264.144073,stockholm
6,2024-11-23,-0.40,0.0,13.910169,280.437408,stockholm
7,2024-11-24,1.35,0.3,18.161278,129.369293,stockholm
8,2024-11-25,1.20,0.0,18.278645,302.124969,stockholm
9,2024-11-26,-1.95,0.0,11.435313,331.821503,stockholm


In [33]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         10 non-null     datetime64[ns]
 1   temperature_2m_mean          10 non-null     float32       
 2   precipitation_sum            10 non-null     float32       
 3   wind_speed_10m_max           10 non-null     float32       
 4   wind_direction_10m_dominant  10 non-null     float32       
 5   city                         10 non-null     object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 448.0+ bytes


In [34]:
air_quality_fg.insert(aq_today_df)  #insert air quality to feature store

2024-11-17 18:47:59,717 INFO: 	5 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1169559/fs/1160262/fg/1348955


Uploading Dataframe: 0.00% |          | Rows 0/1 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: air_quality_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1169559/jobs/named/air_quality_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1757f3ac0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25_2_days_before",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 672781
         }
       },
       "result": {
         "observed_value": 22.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-17T05:47:59.000716Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between

In [35]:
weather_fg.insert(daily_df)     #Insert weather data to feature store

2024-11-17 18:48:09,931 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1169559/fs/1160262/fg/1348956


Uploading Dataframe: 0.00% |          | Rows 0/10 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1169559/jobs/named/weather_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x16bb76dd0>,
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation_sum",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 672784
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 10,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2024-11-17T05:48:09.000930Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between